In [ ]:
"""
Request handler for HTTP requests with retry logic and error handling.
"""

import asyncio
import json
import time
from typing import Dict, Optional, Any, Union

import aiohttp
import requests
from loguru import logger


class RequestHandler:
    """
    Handles HTTP requests with retry logic and error handling.
    
    This class encapsulates the HTTP request functionality, providing
    both synchronous and asynchronous request methods with retry logic.
    """

    def __init__(
        self,
        base_url: str,
        default_timeout: Optional[int] = 30,
        long_timeout: Optional[int] = 300,
        max_retries: Optional[int] = 5,
        retry_delay: Optional[int] = 5,
        semaphore_limit: Optional[int] = 50,
        verify_ssl: Optional[bool] = False,
    ):
        """
        Initialize the request handler.
        
        Args:
            base_url: Base URL for all requests
            default_timeout: Default timeout for requests in seconds
            long_timeout: Timeout for long-running requests in seconds
            max_retries: Maximum number of retry attempts
            retry_delay: Delay between retries in seconds
            semaphore_limit: Maximum number of concurrent requests
            verify_ssl: Whether to verify SSL certificates
        """
        self.base_url = base_url
        self.default_timeout = default_timeout
        self.long_timeout = long_timeout
        self.max_retries = max_retries
        self.retry_delay = retry_delay
        self.semaphore_limit = semaphore_limit
        self.verify_ssl = verify_ssl
        
        # Initialize semaphore for async requests
        self._semaphore = asyncio.Semaphore(semaphore_limit)

    #------------------------------------------------------------------
    # Standard HTTP Methods
    #------------------------------------------------------------------

    def post(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        files: Optional[Dict[str, Any]] = None,
        timeout: Optional[int] = None,
    ) -> Optional[requests.Response]:
        """
        Perform a POST request with retry logic.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            files: Optional files to upload
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if all retries failed
        """
        return self._post_with_retry(
            path=path,
            headers=headers,
            payload=payload,
            files=files,
            retries=self.max_retries,
            timeout=timeout or self.long_timeout,
        )

    def get(
        self, 
        path: str, 
        headers: Dict[str, str], 
        timeout: Optional[int] = None
    ) -> Optional[requests.Response]:
        """
        Perform a GET request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            return requests.get(
                url,
                headers=headers,
                verify=self.verify_ssl,
                timeout=timeout or self.default_timeout,
            )
        except Exception as e:
            logger.error(f"GET request to {url} failed: {str(e)}")
            return None

    def put(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        timeout: Optional[int] = None,
    ) -> Optional[requests.Response]:
        """
        Perform a PUT request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            return requests.put(
                url,
                headers=headers,
                json=payload,
                verify=self.verify_ssl,
                timeout=timeout or self.default_timeout,
            )
        except Exception as e:
            logger.error(f"PUT request to {url} failed: {str(e)}")
            return None

    def delete(
        self, 
        path: str, 
        headers: Dict[str, str], 
        timeout: Optional[int] = None
    ) -> Optional[requests.Response]:
        """
        Perform a DELETE request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            return requests.delete(
                url,
                headers=headers,
                verify=self.verify_ssl,
                timeout=timeout or self.default_timeout,
            )
        except Exception as e:
            logger.error(f"DELETE request to {url} failed: {str(e)}")
            return None

    def post_stream(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        timeout: Optional[int] = None,
    ) -> Optional[requests.Response]:
        """
        Perform a POST request optimized for streaming responses.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object for streaming or None if request failed
        """
        url = self.base_url + path
        
        try:
            # Use stream=True to keep the connection open for streaming
            response = requests.post(
                url,
                headers=headers,
                json=payload,
                stream=True,
                verify=self.verify_ssl,
                timeout=timeout or self.long_timeout,
            )
            
            # Check for HTTP errors
            if response.status_code >= 400:
                logger.error(f"Stream request failed: HTTP {response.status_code}")
                logger.error(f"Error response: {response.text}")
                response.close()
                return None
                
            return response
            
        except Exception as e:
            logger.error(f"Error in streaming request to {url}: {str(e)}")
            return None

    #------------------------------------------------------------------
    # Asynchronous HTTP Methods
    #------------------------------------------------------------------

    async def async_post(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        files: Optional[Dict[str, Any]] = None,
        timeout: Optional[int] = None,
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous POST request with retry logic.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            files: Optional files to upload
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if all retries failed
        """
        return await self._async_post_with_retry(
            path=path,
            headers=headers,
            payload=payload,
            files=files,
            retries=self.max_retries,
            timeout=timeout or self.long_timeout,
        )

    async def async_get(
        self, 
        path: str, 
        headers: Dict[str, str], 
        timeout: Optional[int] = None
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous GET request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            async with self._semaphore:
                async with aiohttp.ClientSession() as session:
                    return await session.get(
                        url,
                        headers=headers,
                        timeout=timeout or self.default_timeout,
                        ssl=None if not self.verify_ssl else True,
                    )
        except Exception as e:
            logger.error(f"Async GET request to {url} failed: {str(e)}")
            return None

    async def async_put(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        timeout: Optional[int] = None,
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous PUT request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            async with self._semaphore:
                async with aiohttp.ClientSession() as session:
                    return await session.put(
                        url,
                        headers=headers,
                        json=payload,
                        timeout=timeout or self.default_timeout,
                        ssl=None if not self.verify_ssl else True,
                    )
        except Exception as e:
            logger.error(f"Async PUT request to {url} failed: {str(e)}")
            return None

    async def async_delete(
        self, 
        path: str, 
        headers: Dict[str, str], 
        timeout: Optional[int] = None
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous DELETE request.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            timeout: Request timeout in seconds (overrides default)
            
        Returns:
            Response object or None if request failed
        """
        url = self.base_url + path
        try:
            async with self._semaphore:
                async with aiohttp.ClientSession() as session:
                    return await session.delete(
                        url,
                        headers=headers,
                        timeout=timeout or self.default_timeout,
                        ssl=None if not self.verify_ssl else True,
                    )
        except Exception as e:
            logger.error(f"Async DELETE request to {url} failed: {str(e)}")
            return None

    async def async_post_stream(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        timeout: Optional[int] = None,
    ) -> Optional[bytes]:
        """
        Perform an asynchronous POST request optimized for streaming responses.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            timeout: Request timeout in seconds
            
        Returns:
            Response content as bytes or None if request failed
        """
        url = self.base_url + path
        client_timeout = aiohttp.ClientTimeout(total=timeout or self.long_timeout)
        
        try:
            async with aiohttp.ClientSession(timeout=client_timeout) as session:
                response = await session.post(
                    url,
                    headers=headers,
                    json=payload,
                    ssl=None if not self.verify_ssl else True,
                    raise_for_status=False
                )
                
                # Check for HTTP errors
                if response.status >= 400:
                    logger.error(f"Stream request failed: HTTP {response.status}")
                    error_body = await response.text()
                    logger.error(f"Error response: {error_body}")
                    return None
                
                # Get the full content for the response
                return await response.read()
                
        except Exception as e:
            logger.error(f"Error in streaming request to {url}: {str(e)}")
            return None

    async def simple_async_stream(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        timeout: Optional[int] = None
    ) -> Optional[str]:
        """
        Perform an async streaming request and process the response in one step.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            timeout: Request timeout in seconds
            
        Returns:
            Final result string or None
        """
        content = await self.async_post_stream(path, headers, payload, timeout)
        if content:
            return await self.process_async_stream_response(content)
        return None

    #------------------------------------------------------------------
    # Response Processing Methods
    #------------------------------------------------------------------

    @staticmethod
    def process_response(response: Any) -> Optional[Any]:
        """
        Process API response with error handling.
        
        Args:
            response: API response
            
        Returns:
            Processed response or None
        """
        if response is None:
            logger.error("Failed to get response after retries")
            return None
            
        try:
            # If response has json method, use it
            if hasattr(response, 'json') and callable(response.json):
                return response.json()
            # If response is already processed, return it
            return response
        except Exception as e:
            logger.error(f"Error parsing response: {e}")
            # Try to return as text if json fails
            try:
                if hasattr(response, 'text'):
                    if callable(response.text):
                        return response.text()
                    return response.text
            except Exception:
                pass
            return None

    @staticmethod
    async def process_async_response(response: Any) -> Optional[Any]:
        """
        Process async API response with error handling.
        
        Args:
            response: Async API response
            
        Returns:
            Processed response or None
        """
        if response is None:
            logger.error("Failed to get async response after retries")
            return None
            
        try:
            # Try to parse as JSON
            return await response.json()
        except Exception as e:
            logger.error(f"Error processing response: {e}")
            
            # Try to return as text if json fails
            try:
                return await response.text()
            except Exception:
                pass
            return None

    @staticmethod
    def process_stream_response(response: requests.Response) -> Optional[str]:
        """
        Process the streaming response from the CMS API.
        
        Args:
            response: The response object from the requests post call
            
        Returns:
            The final answer from the streamed response
        """
        if response is None:
            logger.error("No response object provided for streaming")
            return None
            
        buffer = ""
        try:
            # Use proper chunk iteration with a reasonable chunk size
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    data = chunk.decode("utf-8", errors="ignore")
                    buffer += data
                    
                    # Process complete JSON objects from the buffer
                    while buffer:
                        try:
                            json_data, index = json.JSONDecoder().raw_decode(buffer)
                            buffer = buffer[index:].lstrip()
                            
                            is_streaming = json_data.get("streaming", False)
                            if is_streaming:
                                if "stream_data" in json_data and "text" in json_data["stream_data"]:
                                    streamed_text = json_data["stream_data"]["text"]
                                    print(streamed_text, end="")
                                    time.sleep(1e-6)
                            else:
                                # Final response found
                                if "final_response" in json_data and "answer" in json_data["final_response"]:
                                    final_response = json_data["final_response"]
                                    answer = final_response["answer"]
                                    return answer
                        except json.JSONDecodeError:
                            # Incomplete JSON, wait for more data
                            break
        except Exception as e:
            logger.error(f"Error processing stream: {str(e)}")
        finally:
            # Always close the response
            response.close()
        
        return None

    @staticmethod
    async def process_async_stream_response(content: bytes) -> Optional[str]:
        """
        Process the asynchronous streaming response content from the CMS API.
        
        Args:
            content: The response content as bytes
            
        Returns:
            The final answer from the streamed response
        """
        if content is None:
            logger.error("No response content provided for streaming")
            return None
            
        # Decode the content to text
        try:
            text_content = content.decode("utf-8", errors="ignore")
        except Exception as e:
            logger.error(f"Error decoding streaming content: {e}")
            return None
            
        buffer = text_content
        
        # Process complete JSON objects from the buffer
        while buffer:
            try:
                json_data, index = json.JSONDecoder().raw_decode(buffer)
                buffer = buffer[index:].lstrip()
                
                is_streaming = json_data.get("streaming", False)
                if is_streaming:
                    if "stream_data" in json_data and "text" in json_data["stream_data"]:
                        streamed_text = json_data["stream_data"]["text"]
                        print(streamed_text, end="")
                else:
                    # Final response found
                    if "final_response" in json_data and "answer" in json_data["final_response"]:
                        final_response = json_data["final_response"]
                        answer = final_response["answer"]
                        return answer
            except json.JSONDecodeError:
                # If we can't parse anything from the buffer, exit the loop
                break
        
        # If we get here, we couldn't find the final response
        logger.warning("No final response found in streaming content")
        return None

    #------------------------------------------------------------------
    # Internal Implementation Methods
    #------------------------------------------------------------------

    def _post_with_retry(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        files: Optional[Dict[str, Any]] = None,
        retries: Optional[int] = 5,
        timeout: Optional[int] = 300,
    ) -> Optional[requests.Response]:
        """
        Perform an HTTP POST request with retry logic.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            files: Optional files to upload
            retries: Maximum number of retries
            timeout: Request timeout in seconds
            
        Returns:
            Response object or None if all retries failed
        """
        url = self.base_url + path
        remaining_retries = retries
        
        while remaining_retries > 0:
            try:
                # Handle payload differently if files are provided to avoid conflicts
                if files is not None:
                    response = requests.post(
                        url,
                        headers=headers,
                        data=payload,  # Use data instead of json when files are present
                        files=files,
                        verify=self.verify_ssl,
                        timeout=timeout,
                    )
                else:
                    response = requests.post(
                        url,
                        headers=headers,
                        json=payload,  # Use json for proper serialization
                        verify=self.verify_ssl,
                        timeout=timeout,
                    )
                
                # Check for HTTP errors
                if response.status_code >= 300:
                    detail = f"Error code from API: {response.status_code} {response.text}"
                    logger.error(f"Request failed: {detail}")
                    
                    # For 5xx errors, retry. For 4xx, return the response
                    if response.status_code < 500:
                        return response
                else:
                    # Success
                    return response
                    
            except Exception as e:
                logger.error(f"Exception during request to {url}: {str(e)}")
            
            # If we get here, we need to retry
            remaining_retries -= 1
            if remaining_retries > 0:
                logger.info(
                    f"Retrying {url} in {self.retry_delay} seconds. "
                    f"Attempts remaining: {remaining_retries}"
                )
                time.sleep(self.retry_delay)
        
        # All retries failed
        logger.error(f"Failed to POST to {url} after {retries} attempts")
        return None

    async def _async_post_with_retry(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        files: Optional[Dict[str, Any]] = None,
        retries: Optional[int] = 5,
        timeout: Optional[int] = 300,
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous HTTP POST request with retry logic.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            files: Optional files to upload
            retries: Maximum number of retries
            timeout: Request timeout in seconds
            
        Returns:
            Response object or None if all retries failed
        """
        url = self.base_url + path
        remaining_retries = retries
        error = None
        
        while remaining_retries > 0:
            try:
                async with self._semaphore:
                    async with aiohttp.ClientSession() as session:
                        # Handle both JSON payload and form data with files
                        if files:
                            # aiohttp handles multipart form data for files
                            form_data = aiohttp.FormData()
                            
                            # Add any payload fields
                            if payload:
                                for key, value in payload.items():
                                    form_data.add_field(key, str(value))
                            
                            # Add files
                            for key, file_info in files.items():
                                form_data.add_field(
                                    key,
                                    file_info[1],  # File content
                                    filename=file_info[0],  # Filename
                                    content_type=file_info[2]  # Content type
                                )
                                
                            response = await session.post(
                                url,
                                headers=headers,
                                data=form_data,
                                timeout=timeout,
                                ssl=None if not self.verify_ssl else True,
                            )
                        else:
                            # Regular JSON payload
                            response = await session.post(
                                url,
                                headers=headers,
                                json=payload,
                                timeout=timeout,
                                ssl=None if not self.verify_ssl else True,
                            )
                        
                        # Check for HTTP errors
                        if response.status >= 300:
                            detail = f"Error code from API: {response.status} {await response.text()}"
                            logger.error(f"Request failed: {detail}")
                            
                            # For 5xx errors, retry. For 4xx, return the response
                            if response.status < 500:
                                return response
                        else:
                            # Success
                            return response
            
            except Exception as e:
                error = e
                logger.error(
                    f"Error occurred calling API {url}. Details: {str(e)}. "
                    f"Current retry attempt# {retries - remaining_retries + 1} of {retries}."
                )
                
            # If we get here, we need to retry
            remaining_retries -= 1
            if remaining_retries > 0:
                logger.info(f"Sleeping for {self.retry_delay} seconds before next retry")
                await asyncio.sleep(self.retry_delay)
        
        # All retries failed
        logger.error(f"Failed to POST to {url} after {retries} attempts")
        if error:
            logger.error(f"Last error: {str(error)}")
            
        return None